# 图像分类任务

## 图像分类数据集的引入

In [1]:
import torch
import torchvision

In [2]:
def loadFashion(batchSize:int = 256, reSize:int = False, numWorkers:int =4, isDownloaded:bool = True):
    trans = []
    if reSize:
        trans.append(torchvision.transforms.Resize(reSize))
    trans.append(torchvision.transforms.ToTensor())
    trans = torchvision.transforms.Compose(trans)
    trainData = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=trans, download=isDownloaded)
    testData = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=trans, download=isDownloaded)
    return (torch.utils.data.DataLoader(trainData, batchSize, shuffle=True, num_workers=numWorkers), torch.utils.data.DataLoader(testData, batchSize, shuffle=False, num_workers=numWorkers))

In [3]:
trainData, testData = loadFashion()

## Softmax回归

### 初始化模型与参数

In [4]:
net = torch.nn.Sequential(torch.nn.Flatten(), torch.nn.Linear(784, 10))
net[1].weight.data.normal_(mean=0., std=0.1)
net[1].bias.data.fill_(0.)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

### 定义损失函数

In [5]:
loss = torch.nn.CrossEntropyLoss(reduction='none')

### 优化算法

In [6]:
trainer = torch.optim.SGD(net.parameters(), lr=0.1)

### 训练模型

In [7]:
epochs = 50
for epoch in range(epochs):
    for X, y in trainData:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.sum().backward()      # Don't forget sum()
        trainer.step()


### 判断正确率

In [8]:
right = 0
count = 0
for X, y in testData:
    right += (net(X).argmax(axis=1)==y).sum()
    count += y.numel()
print(right/count)

tensor(0.8231)
